<center>    
    <h1 id='conditional-probability-notebook-3' style='color:#7159c1; font-size:350%'>Hidden Markov Model (HMM)</h1>
    <i style='font-size:125%'>Calculating Probabilities Through Conditions</i>
</center>

> **Topics**

```
- 📦 Hidden Markov Model (HMM)
```

In [1]:
# ---- Imports ----
import numpy as np
from IPython.display import HTML

# ---- Constants ----
VIDEOS_WIDTH = (600)
VIDEOS_PATH = ('./videos')

# ---- Functions ----
def generateVideoEmbed(path, width):
    """
    Generates a string containing a centered video tag with a specific width and video source.

    - Input:
        / path: string;
        / width: float.

    - Output:
        / video_tag: string.
    """
    video_tag = f'<center><video width="{width}" autoplay controls loop><source src="{path}" type="video/mp4" />Your browser does not support the video tag 😢</video></center>'
    return video_tag

<h1 id='0-hidden-markov-model-hmm' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>📦 | Hidden Markov Model (HMM)</h1>

Picture a professor that can be either happy or sad daily and we desire to guess his mood given the color of his shirt. After a few months of study, we analyzed the following info:

**Mood Probabilities:**

- the probability of the professor be happy given he was happy last day is `70.00%`;
- the probability of the professor be sad given he was happy last day is `30.00%`;
- the probability of the professor be happy given he was sad last day is `50.00%`;
- the probability of the professor be sad given he was sad last day is `50.00%`.

**Shirt Color Probabilities:**

- the probability of the professor wears a red shirt given he is happy is `80.00%`;
- the probability of the professor wears a green shirt given he is happy is `10.00%`;
- the probability of the professor wears a blue shirt given he is happy is `10.00%`;
- the probability of the professor wears a red shirt given he is sad is `20.00%`;
- the probability of the professor wears a green shirt given he is sad is `30.00%`;
- the probability of the professor wears a blue shirt given he is sad is `50.00%`.

**Initial Probabilities:**

- the probability of the professor be happy in the first day of the analysis is `40.00%`;
- the probability of the professor be sad in the first day of the analysis is `60.00%`.

---

Sorting all this data into tables, we have:

**Mood Probabilities Table:**

|       | Happy | Sad  |
| ---   | ---   | ---  |
| Happy | 0.70  | 0.30 |
| Sad   | 0.50  | 0.50 |

**Shirt Color Probabilities Table:**

|       | Red | Green  | Blue |
| ---   | ---   | ---  | ---  |
| Happy | 0.80  | 0.10 | 0.10 |
| Sad   | 0.20  | 0.30 | 0.50 |

**Initial Probabilities Table:**

| Happy | Sad  |
| ---   | ---  |
| 0.40  | 0.60 |

---

Illustrating all the sorted data into a diagram, also known as `Hidden Markov Chain`, we have:

In [4]:
# ---- Hidden Markov Model (HMM): Visualization ----
HTML(generateVideoEmbed(f'{VIDEOS_PATH}/13-HiddenMarkovModel.mp4', VIDEOS_WIDTH))

Ok, since we already have our problem well structured, consider that the professor wore a `green shirt`, a `blue shirt` and a `red shirt` respectively in three days and we desire to know his daily mood over this period.

In order to achieve it, we must calculate the `Maximum Probability` of all combinations of our varibles: mood and color shirt. Let's call the first variable as `M` and the second one as `C`, so:

- **M1, M2, M3** - `corresponds to the professor's mood in each of the three days`;
- **C1, C2, C3** - `corresponds to the professor's shirt color in each of the three days`.

So, the `Maximum Probability` is given by:

$$
\text{Max}(\mathbf{M1}, \mathbf{M2}, \mathbf{M3}) =
\mathbf{P}{
    (
        \mathbf{C1} = \text{Green},
        \mathbf{C2} = \text{Blue},
        \mathbf{C3} = \text{Red},
        \\
        \mathbf{M1} = \text{Happy or Sad},
        \mathbf{M2} = \text{Happy or Sad},
        \mathbf{M3} = \text{Happy or Sad}
    )
}
$$

The probability of our variables is given by the produtc of the probability of each outcome given the other ones:

$$
\begin{array}
    \mathbf{P}{
        (
            \mathbf{C1} = \text{Green},
            \mathbf{C2} = \text{Blue},
            \mathbf{C3} = \text{Red},
            \\
            \mathbf{M1} = \text{Happy or Sad},
            \mathbf{M2} = \text{Happy or Sad},
            \mathbf{M3} = \text{Happy or Sad}
        )
    } = \\
    \mathbf{P}(\mathbf{C3} | \mathbf{C2}, \mathbf{C1}, \mathbf{M3}, \mathbf{M2}, \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{C2} | \mathbf{C1}, \mathbf{M3}, \mathbf{M2}, \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{C1} | \mathbf{M3}, \mathbf{M2}, \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{M3} | \mathbf{M2}, \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{M2} | \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{M1})
\end{array}
$$

And finally, since the shirt color is only influenced by the professor's current mood, and the professor's current mood is influenced by his mood on the previous day, we can simplify our equation to:

$$
\begin{array}
    \mathbf{P}{
        (
            \mathbf{C1} = \text{Green},
            \mathbf{C2} = \text{Blue},
            \mathbf{C3} = \text{Red},
            \\
            \mathbf{M1} = \text{Happy or Sad},
            \mathbf{M2} = \text{Happy or Sad},
            \mathbf{M3} = \text{Happy or Sad}
        )
    } = \\
    \mathbf{P}(\mathbf{C3} | \mathbf{M3}) \times \\
    \mathbf{P}(\mathbf{C2} | \mathbf{M2}) \times \\
    \mathbf{P}(\mathbf{C1} | \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{M3} | \mathbf{M2}) \times \\
    \mathbf{P}(\mathbf{M2} | \mathbf{M1}) \times \\
    \mathbf{P}(\mathbf{M1} | \mathbf{S})
\end{array}
$$

Where $\mathbf{S}$ is the professor's mood in the initial state!

We won't calculate the probability of all combinations of the events outcomes, since it would take a loooong time and space in this notebook and the goal here is to give you the notion of how `Hidden Markov Model` is structured, implemented and calculated. Oh, about the answer of professor's moods over the three days? After doing all the calculations, we will get the that the higher probability is that the professor was `{S, S, H}`, that is:

- `sad` in the first day;
- `sad` in the second day;
- `happy` in the third day.

---

In a nutshell, Hidden Markov Model (HMM) consists figuring out the `hidden states (happy and sad in our example)` given the `observations (red, blue and green shirt in our example)` and the outcome probabilities of the occurrence of each hidden state and observation.

See you in the next topic: `Calculus`!!

---

<h1 id='reach-me' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>📫 | Reach Me</h1>

> **Email** - [csfelix08@gmail.com](mailto:csfelix08@gmail.com?)

> **Linkedin** - [linkedin.com/in/csfelix/](https://www.linkedin.com/in/csfelix/)

> **GitHub:** - [CSFelix](https://github.com/CSFelix)

> **Kaggle** - [DSFelix](https://www.kaggle.com/dsfelix)

> **Portfolio** - [CSFelix.io](https://csfelix.github.io/).